<a href="https://colab.research.google.com/github/incubusV/colab/blob/main/Walkability_Score_Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import osmnx as ox
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import math
import warnings

def normalize(value, min_val=0, max_val=100):
    """Normalize a value to a 0–100 scale based on a fixed range."""
    if value < min_val or value > max_val:
        warnings.warn(f"Value {value} is outside the expected range [{min_val}, {max_val}].")
    return min(100, max(0, (value - min_val) / (max_val - min_val) * 100))

def calculate_walkability_score(location, radius=1000, street_weight=0.4, intersection_weight=0.4, amenity_weight=0.2):
    """
    Calculate a walkability score for a location using:
    - Street network density (total street length per km²)
    - Intersection density (nodes with degree > 1 per km²)
    - Amenity density (number of amenities per km²)
    
    Parameters:
        location: Tuple or list (latitude, longitude) representing the center point.
        radius: Radius (in meters) to consider around the location.
        street_weight: Weight for street density (default 0.4).
        intersection_weight: Weight for intersection density (default 0.4).
        amenity_weight: Weight for amenity density (default 0.2).
    
    Returns:
        walkability_score: The weighted walkability score.
        details: A dictionary with raw metric values.
    """
    try:
        # Download the street network (walking network) and amenities from OpenStreetMap
        G = ox.graph_from_point(location, dist=radius, network_type='walk')
        amenities = ox.geometries_from_point(location, tags={'amenity': ['cafe', 'restaurant', 'shop', 'school', 'hospital']}, dist=radius)
        
        # Calculate the area in km² for a circle with the given radius.
        area = math.pi * (radius ** 2) / 1_000_000
        
        # Calculate street density:
        edge_lengths = [data.get('length', 0) for u, v, data in G.edges(data=True)]
        total_length_km = sum(edge_lengths) / 1000
        street_density = total_length_km / area if area > 0 else 0
        
        # Calculate intersection density:
        nodes, _ = ox.graph_to_gdfs(G)
        nodes['street_count'] = nodes.index.map(G.degree)
        intersection_density = len(nodes[nodes['street_count'] > 2]) / area if area > 0 else 0
        
        # Calculate amenity density:
        amenity_density = len(amenities) / area if area > 0 else 0
        
        # Combine the metrics into a weighted walkability score:
        walkability_score = (
            street_weight * normalize(street_density) +
            intersection_weight * normalize(intersection_density) +
            amenity_weight * normalize(amenity_density)
        )
        
        return walkability_score, {
            'street_density': street_density,
            'intersection_density': intersection_density,
            'amenities_per_sqkm': amenity_density
        }
    except Exception as e:
        print(f"Error calculating walkability score: {e}")
        return None, None

# Example usage:
if __name__ == '__main__':
    # Example: Centered at a location (latitude, longitude)
    location = (40.748817, -73.985428)  # Example: New York, Empire State Building area
    score, details = calculate_walkability_score(location, radius=1000)
    if score is not None:
        print("Walkability Score:", score)
        print("Details:", details)